<a href="https://colab.research.google.com/github/ColeJMitchell/machine-learning-stock-prediction/blob/main/trade_simulation_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import numpy as np
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
import seaborn as sns
import datetime, time
from termcolor import colored
from IPython.display import clear_output
!pip install tensorflow
from tensorflow.keras.models import load_model
from sklearn.preprocessing import MinMaxScaler
clear_output()

## Part _: _

In [5]:
# Read in the results from model predictions
TESTDATADIR = "data/test_stocks/"
TESTSTOCKS = [
    "AAPL", # Apple
    "AMD",  # AMD
    "AMZN", # Amazon
    "META", # Meta Platforms
    "NFLX", # Netflix
    "QCOM", # Qualcomm
    "SBUX", # Starbucks
    "SCSO", # Cisco
    "TSLA", # Tesla
]
predictions = {}
for stock in TESTSTOCKS:
    predictions[stock] = pd.read_parquet(TESTDATADIR + stock + "_results.parquet")

In [8]:
#Load the LSTM model and simulate it by paper trading
model = load_model("./saved_models/stock_prediction.h5", compile = False)
model.summary()

E0000 00:00:1746989991.134654   12191 cuda_executor.cc:1228] INTERNAL: CUDA Runtime error: Failed call to cudaGetRuntimeVersion: Error loading CUDA libraries. GPU will not be used.: Error loading CUDA libraries. GPU will not be used.
W0000 00:00:1746989991.135320   12191 gpu_device.cc:2341] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 30, 64)         │        16,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 30, 64)         │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation (Activation)         │ (None, 30, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 32)             │        12,416 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 32)             │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_1 (Activation)       │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 16)             │           528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 16)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 16)             │           272 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 16)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            17 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 30,513 (119.19 KB)

 Trainable params: 30,321 (118.44 KB)

 Non-trainable params: 192 (768.00 B)

Part _: sentiment scorse

In [9]:
# Read in the combined reddit semtiment data
file = "merged-reddit-wsb-wss.parquet"
reddit_data = pd.read_parquet(f"./data/{file}")

display(reddit_data.head())

,title,upvote_ratio,id,permalink,num_comments,created_utc,selftext,ss_neg,ss_neu,ss_pos,ss_compound
0,Nivea Along,0.67,1k0t4jk,/r/wallstreetbets/comments/1k0t4jk/nivea_along/,5,1.744832e+09,After -7% yesterday and -10% today,0.12,0.805,0.076,-0.6908
1,Powell to Volatile Stock Market: You’re on You...,0.86,1k0unbq,/r/wallstreetbets/comments/1k0unbq/powell_to_v...,2,1.744836e+09,,0.12,0.805,0.076,-0.6908
2,Made back the last Wendy’s paycheck I lost,0.94,1k0tv2y,/r/wallstreetbets/comments/1k0tv2y/made_back_t...,6,1.744834e+09,,0.12,0.805,0.076,-0.6908
3,After market observation. When I finished buyi...,0.72,1k0tnqx,/r/wallstreetbets/comments/1k0tnqx/after_marke...,8,1.744833e+09,https://preview.redd.it/41ilvj6f39ve1.png?widt...,0.12,0.805,0.076,-0.6908
4,Ominous,0.85,1k0thnd,/r/wallstreetbets/comments/1k0thnd/ominous/,110,1.744833e+09,NVIDIA 2024 is starting to rhyme like Cisco 20...,0.12,0.805,0.076,-0.6908


## Part _: Formatting the data so that it can be read by the model

In [ ]:
# Data Format
print(predictions['AAPL'])

          Date  Actual_Scaled  Prediction_Scaled  Actual  Prediction
0   2025-03-25       0.592725           0.532328  223.75  218.519653
1   2025-03-24       0.557852           0.553596  220.73  220.361404
2   2025-03-21       0.529446           0.521709  218.27  217.599976
3   2025-03-20       0.481293           0.500552  214.10  215.767776
4   2025-03-19       0.494457           0.464448  215.24  212.641190
..         ...            ...                ...     ...         ...
215 2024-05-14       0.173326           0.243720  187.43  193.526138
216 2024-05-13       0.160046           0.241477  186.28  193.331909
217 2024-05-10       0.122748           0.240314  183.05  193.231171
218 2024-05-09       0.140300           0.237339  184.57  192.973541
219 2024-05-08       0.119169           0.238530  182.74  193.076721

[220 rows x 5 columns]


In [ ]:
prediction_windows = {}
actual_values = {}
window_size = 30

# Stocks need to remain separated for the paper trading
for stock in TESTSTOCKS:
    prediction_windows[stock] = []
    actual_values[stock] = []
    for i in range(0, 220 - window_size + 1, window_size):
        window = predictions[stock]["Prediction"][i:i + window_size]
        prediction_windows[stock].append(np.array(window))
        actual_values[stock].append(predictions[stock]["Actual"][i+window_size])

print(f"Shape of Stock Sliding Windows: {np.array(prediction_windows[TESTSTOCKS[0]]).shape}")
print(f"Shape of Corresponding Actual Value Array: {np.array(actual_values[TESTSTOCKS[0]]).shape}")


Shape of Stock Sliding Windows: (7, 30)
Shape of Corresponding Actual Value Array: (7,)


In [ ]:
# A list of stocks purchased by the LSTM model is saved so it can know when to sell, the model starts with the same amount of money as the random model - 10,000 dollars
portfolio = {}
initial_wallet = 10000

for stock in TESTSTOCKS:
  portfolio[stock] = {"Holding": 1000, "Bought": 0, "Purchase_Prices": []}
# If a stock is predicted to rise in the future (larger value then final entry in sliding windo), it is bought by the model, half of the allocated money for the stock is bought (1000 dollars)
# If the stock is predicted to fall below the purchase price the n
for stock in TESTSTOCKS:
  windows = prediction_windows[stock]
  for window in windows:
    baseline_price = window[-1]
    scaler = MinMaxScaler(feature_range=(0, 1))
    normalized_window = scaler.fit_transform(window.reshape(-1,1))
    print(normalized_window)
    prediction = model.predict(normalized_window.reshape(30,1), verbose=0)
    print(prediction)
    current_holdings = portfolio[stock]["Holding"]
    current_bought = portfolio[stock]["Bought"]
    if baseline_price < prediction[0].item():
      if current_holdings - 100 <= 0:
        continue
      else:
        # Stocks are bought in 100 dollar increments
        portfolio[stock]["Holdings"] =  portfolio[stock]["Holdings"] - 100
        portfolio[stock]["Bought"] = portfolio[stock]["Bought"] + 100
        portfolio[stock]["Purchase_Prices"].append(baseline_price)
    elif baseline_price > prediction.item():
      if current_bought <= 0:
        continue
      else:
        for i in range (len(portfolio[stock]["Purchase_Prices"])):
          # Sells the bought stock at the new price if it is worse than one of the previously purchase price points
          if portfolio[stock]["Purchase_Prices"][i] > prediction.item():
            reduced_percentage = prediction / portfolio[stock]["Purchase_Prices"][i]
            portfolio[stock]["Holdings"] =  portfolio[stock]["Holdings"] + (100 * reduced_percentage)
            portfolio[stock]["Bought"] = portfolio[stock]["Bought"] - 100
            del portfolio[stock]["Purchase_Prices"][i]
            break
    else:
      pass

[[0.3370242 ]
 [0.40401268]
 [0.30357313]
 [0.2369318 ]
 [0.12321091]
 [0.14712954]
 [0.07855892]
 [0.10619926]
 [0.08883619]
 [0.        ]
 [0.17006159]
 [0.27648592]
 [0.49780083]
 [0.8287134 ]
 [0.72972393]
 [0.74310493]
 [0.73390007]
 [0.7901468 ]
 [0.8850708 ]
 [0.7723379 ]
 [0.85228825]
 [0.99041843]
 [1.        ]
 [0.9798231 ]
 [0.9839182 ]
 [0.96462727]
 [0.9561329 ]
 [0.9566145 ]
 [0.88849545]
 [0.7625532 ]]
[[0.32922456]
 [0.4105609 ]
 [0.277398  ]
 [0.25561562]
 [0.22846907]
 [0.2339477 ]
 [0.22099712]
 [0.22639082]
 [0.22341555]
 [0.2014982 ]
 [0.2402478 ]
 [0.25735384]
 [0.5094893 ]
 [0.6265341 ]
 [0.59885246]
 [0.6031461 ]
 [0.60020626]
 [0.61713195]
 [0.63799155]
 [0.6120515 ]
 [0.6316429 ]
 [0.6526581 ]
 [0.6535891 ]
 [0.6515553 ]
 [0.6519908 ]
 [0.64983535]
 [0.6488013 ]
 [0.6488613 ]
 [0.63860404]
 [0.60914665]]


ValueError: can only convert an array of size 1 to a Python scalar